In [1]:
!pip install datasets
!pip install transformers
!pip install faiss-cpu

distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 8.1 MB 7.9 MB/s eta 0:00:01     |█████████████████████           | 5.3 MB 1.6 MB/s eta 0:00:02     |██████████████████████▌         | 5.7 MB 1.6 MB/s eta 0:00:02
distutils: /opt/conda/include/python3.7m/UNKNOWN
sysconfig: /opt/conda/include/python3.7m
use

In [22]:
from tqdm import tqdm, trange
import argparse
import random
from transformers import AutoTokenizer
import numpy as np
import torch
from datasets import load_dataset
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup
from torch.utils.data import (DataLoader, SequentialSampler, TensorDataset)

torch.manual_seed(2021)
torch.cuda.manual_seed(2021)
np.random.seed(2021)
random.seed(2021)

In [8]:
dataset = load_dataset('squad_kor_v1')
corpus = list(set(example['context'] for example in dataset['train']))
model_checkpoint = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


In [9]:
sample_idx = np.random.choice(range(len(dataset['train'])),128)
training_dataset = dataset['train'][sample_idx]

q_seqs = tokenizer(training_dataset['question'],padding='max_length',truncation=True,return_tensors='pt')
p_seqs = tokenizer(training_dataset['context'],padding='max_length',truncation=True,return_tensors='pt')

train_dataset = TensorDataset(p_seqs['input_ids'],p_seqs['attention_mask'],p_seqs['token_type_ids'],
                             q_seqs['input_ids'],q_seqs['attention_mask'],q_seqs['token_type_ids'])

In [10]:
def train(args, dataset, p_model, q_model):
    # DataLoader
    train_sampler = RandomSampler(dataset)
    train_dataloader = DataLoader(dataset, sampler = train_sampler, batch_size= args.per_device_train_batch_size)
    
    #Optimizer
    
    no_decay = ['bias','LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params':[p for n,p in p_model.named_parameters() if not any(nd in n for nd in no_decay)],'weight_decay':args.weight_decay},
        {'params':[p for n,p in p_model.named_parameters() if any(nd in n for nd in no_decay)],'weight_decay':0.0},
        {'params':[p for n,p in q_model.named_parameters() if not any(nd in n for nd in no_decay)],'weight_decay':args.weight_decay},
        {'params':[p for n,p in q_model.named_parameters() if any(nd in n for nd in no_decay)],'weight_decay':0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters,lr = args.learning_rate, eps=args.adam_epsilon)
    t_total = len(train_dataloader) // args.gradient_accumulation_steps*args.num_train_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=args.warmup_steps,num_training_steps=t_total)
    
    #Bert training
    global_step =0
    p_model.zero_grad()
    q_model.zero_grad()
    
    torch.cuda.empty_cache()
    
    train_iterator = trange(int(args.num_train_epochs),desc='Epoch')
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader,desc ='Iteration')
        for step, batch in enumerate(epoch_iterator):
            q_encoder.train()
            p_encoder.train()
            
            if torch.cuda.is_available():
                batch = tuple(t.cuda() for t in batch)
                
            p_inputs = {'input_ids':batch[0],
                       'attention_mask':batch[1],
                       'token_type_ids':batch[2]}
            q_inputs = {'input_ids':batch[3],
                       'attention_mask':batch[4],
                       'token_type_ids':batch[5]}
            
            p_outputs = p_model(**p_inputs) #(batch_size,emb_dim)
            q_outputs = q_model(**q_inputs) #(batch_size,emb_dim)
            
            #Calculate similarity score & loss
            
            sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs,0,1))
            
            # target
            
            targets = torch.arange(0,args.per_device_train_batch_size).long()
            if torch.cuda.is_available():
                targets=targets.to('cuda')
                
            sim_scores = F.log_softmax(sim_scores,dim=1)
            loss = F.nll_loss(sim_scores,targets)
            print(loss)
            loss.backward()
            optimizer.step()
            scheduler.step()
            q_model.zero_grad()
            p_model.zero_grad()
            global_step += 1
            
            torch.cuda.empty_cache()
            
        return p_model,q_model

In [11]:
args = TrainingArguments(
        output_dir='dense_retireval',
        evaluation_strategy='epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=2,
        weight_decay=0.01
)

In [13]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self,config):
        super(BertEncoder,self).__init__(config)
        
        self.bert = BertModel(config)
        self.init_weights()
        
    def forward(self,input_ids,attention_mask=None,token_type_ids=None):
        outputs = self.bert(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        return pooled_output


In [14]:
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
    p_encoder.cuda()
    q_encoder.cuda()
    print('GPU enabled')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias',

GPU enabled


In [15]:
p_encoder,q_encoder = train(args,train_dataset,p_encoder,q_encoder)

Iteration:   0%|          | 0/32 [00:00<?, ?it/s]

tensor(21.4702, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:   3%|▎         | 1/32 [00:00<00:20,  1.49it/s]

tensor(9.1994, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:   6%|▋         | 2/32 [00:01<00:17,  1.67it/s]

...


Iteration:  94%|█████████▍| 30/32 [00:13<00:00,  2.21it/s]

tensor(1.5581, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:  97%|█████████▋| 31/32 [00:14<00:00,  2.21it/s]

tensor(1.7951, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration: 100%|██████████| 32/32 [00:14<00:00,  2.17it/s]


In [19]:
search_corpus = list(set(example['context'] for example in dataset['validation']))
len(search_corpus)

960

In [26]:
valid_p_seqs = tokenizer(search_corpus,padding='max_length',truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(valid_p_seqs['input_ids'],valid_p_seqs['attention_mask'],valid_p_seqs['token_type_ids'])
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler,batch_size=2)

p_embs = []

with torch.no_grad():
    
    epoch_iterator = tqdm(valid_dataloader,desc='Iteration',position=0,leave=True)
    p_encoder.eval()

    for _,batch in enumerate(epoch_iterator):
        batch=tuple(t.cuda() for t in batch)
        p_inputs = {'input_ids':batch[0],
                   'attention_mask':batch[1],
                   'token_type_ids':batch[2]}
        outputs = p_encoder(**p_inputs).to('cpu').numpy()
        p_embs.extend(outputs)
        
p_embs = np.array(p_embs)
p_embs.shape

Iteration: 100%|██████████| 480/480 [00:11<00:00, 40.53it/s]


(960, 768)

In [32]:
sample_idx = np.random.choice(range(len(dataset['validation'])),5)
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

In [33]:
query

['흥신학원 측의 기부종용에 교사들이 불법성 여부를 질의한 기관은?',
 '손석희가 미국 유학 뒤 앵커로 복직한 프로그램 이름은?',
 '나경원 의원의 비서로 중학생에 대한 막말과 협박으로 논란이 된 인물은?',
 '바이오쇼크가 최고의 엑스박스 360게임에 뽑힌 연도는?',
 '임시광고경영관리방법은 어느 규정에 속하는가?']

In [36]:
valid_q_seqs = tokenizer(query,padding='max_length',truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
    q_encoder.eval()
    q_embs = q_encoder(**valid_q_seqs).to('cpu').numpy()
    
torch.cuda.empty_cache()
q_embs.shape

(5, 768)

In [37]:
if torch.cuda.is_available():
    p_embs_cuda = torch.Tensor(p_embs).to('cuda')
    q_embs_cuda = torch.Tensor(q_embs).to('cuda')


In [40]:
import time
start_time = time.time()

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda,0,1))

rank = torch.argsort(dot_prod_scores,dim=1,descending=True).squeeze()

print(rank)

print('---------%s second' % (time.time()-start_time))

tensor([[688,  37,   1,  ..., 330, 695, 349],
        [688,  37, 346,  ..., 950, 695, 349],
        [390, 522, 954,  ..., 743, 204, 360],
        [ 37,   1, 688,  ..., 437, 695, 349],
        [  1, 688, 486,  ..., 690, 437, 349]], device='cuda:0')
---------0.0031676292419433594 second


In [42]:
k = 5
for i, q in enumerate(query[:1]):
    print('Search query',q,'\n')
    print('ground truth passage')
    print(ground_truth[i],'\n')
    
    r = rank[i]

    for j in range(k):
        print("top-%d passage with score %.4f" % (j+1,dot_prod_scores[i][r[j]]))
        print(search_corpus[r[j]])

Search query 흥신학원 측의 기부종용에 교사들이 불법성 여부를 질의한 기관은? 

ground truth passage
이후 오마이뉴스는 나경원이 중앙선거관리위원회에 제출한 2004년 정치자금 수입명세서를 입수해 확인한 결과 거기에 일부 기부자의 직업이 교원으로 표시되어 있다는 사실과 해당 교원들의 전화번호가 흥신학원 소속 학교의 국번과 일치한다는 점을 확인했다고 보도했다. 오마이뉴스는 또한 2004년 당시 흥신학원 재단 소속 학교에 근무하던 교사들로부터 학교측의 기부 종용이 있었다는 증언도 나왔다고 보도했다. 이때 여러 교사들이 나경원 의원에게 정치후원금을 냈는데, 일부 교사들은 교사의 정치후원금은 불법이라고 알고 있었기 때문에 교육부에 질의를 했고 이에 대해 교과부는 초중등교원은 정치기부행위를 할 수 없다는 내용의 회신을 보내왔다. 이처럼 일개 교사도 정치후원금을 내는 것이 불법이라는 사실을 알고 있었는데 판사출신인 나경원 의원이 불법인줄 몰랐겠냐는 의혹을 제기하였다. 한편, 나경원은 2011년 10월 20일 KBS라디오와의 인터뷰에서는 2004년 당시에는 공무원들도 선거 정치자금을 내는 것이 금지되지 않았었다고 주장했다. 하지만 오마이뉴스가 교과부에 확인한 바에 의하면 이는 사실과 다르다고 한다. 교과부 교원복무 담당자는 전화통화에서 교원의 정치자금 기부를 금지하는 규정은 1949년 제정된 이래 바뀐 적이 없었고 따라서 교사의 정치자금 기부가 합법이었던 적은 없었다고 밝혔다. 

top-1 passage with score 28.2475
1980년 봄, 그는 김대중, 김종필 등과 대권을 놓고 경쟁하였다. 그는 전두환과 신군부의 쿠데타를 그리 걱정하지 않았고, 결국 5·17 쿠데타로 좌절되었다. 1980년 5월 17일 오전 10시, 김영삼은 신군부 군인들에 의해 가택연금을 당했다. 김영삼은 5월 20일 상도동 자택에서 5.17 비상계엄 확대 조치를 내린 신군부를 규탄하는 기자회견을 열었다. 그는 "오늘 계엄통치를 확대 강화한 5 ·17 사태를, 민

In [43]:
import faiss

num_clusters = 16
niter =5
k = 5

emb_dim = p_embs.shape[-1]
index_flat = faiss.IndexFlatL2(emb_dim)

clus = faiss.Clustering(emb_dim,num_clusters)
clus.verbose = True
clus.niter=niter
clus.train(p_embs,index_flat)

centroids = faiss.vector_float_to_array(clus.centroids)
centroids = centroids.reshape(num_clusters,emb_dim)

quantizer = faiss.IndexFlatL2(emb_dim)
quantizer.add(centroids)

In [44]:
indexer = faiss.IndexIVFScalarQuantizer(quantizer, quantizer.d, quantizer.ntotal, faiss.METRIC_L2)
indexer.train(p_embs)
indexer.add(p_embs)

In [45]:
start_time = time.time()
D,I = indexer.search(q_embs,k)

print('------------%s seconds---' % (time.time()-start_time))

------------0.0031926631927490234 seconds---


In [46]:
print('--distance--')
print(D)
print('\n')
print('index of top 5 passage')
print(I)

--distance--
[[16.545177 16.620083 17.717316 17.736902 18.45747 ]
 [13.102026 13.667351 14.499323 14.800053 14.920881]
 [19.582127 20.353817 21.163902 21.78476  21.880838]
 [15.566828 16.145767 17.03789  17.187622 17.29128 ]
 [23.740076 24.322138 25.140816 25.427517 25.784266]]


index of top 5 passage
[[606 886 950 927  57]
 [606 886 950 927  57]
 [606 886 950  57 927]
 [606 886 950  57 927]
 [606 886 950 927  57]]


In [48]:
for i, q in enumerate(query[:1]):
    print('Search query',q,'\n')
    print('ground truth passage')
    print(ground_truth[i],'\n')
    
    d = D[i]
    i = I[i]

    for j in range(k):
        print("top-%d passage with score %.4f" % (j+1,d[j]))
        print(search_corpus[i[j]])
    print('\n')

Search query 흥신학원 측의 기부종용에 교사들이 불법성 여부를 질의한 기관은? 

ground truth passage
이후 오마이뉴스는 나경원이 중앙선거관리위원회에 제출한 2004년 정치자금 수입명세서를 입수해 확인한 결과 거기에 일부 기부자의 직업이 교원으로 표시되어 있다는 사실과 해당 교원들의 전화번호가 흥신학원 소속 학교의 국번과 일치한다는 점을 확인했다고 보도했다. 오마이뉴스는 또한 2004년 당시 흥신학원 재단 소속 학교에 근무하던 교사들로부터 학교측의 기부 종용이 있었다는 증언도 나왔다고 보도했다. 이때 여러 교사들이 나경원 의원에게 정치후원금을 냈는데, 일부 교사들은 교사의 정치후원금은 불법이라고 알고 있었기 때문에 교육부에 질의를 했고 이에 대해 교과부는 초중등교원은 정치기부행위를 할 수 없다는 내용의 회신을 보내왔다. 이처럼 일개 교사도 정치후원금을 내는 것이 불법이라는 사실을 알고 있었는데 판사출신인 나경원 의원이 불법인줄 몰랐겠냐는 의혹을 제기하였다. 한편, 나경원은 2011년 10월 20일 KBS라디오와의 인터뷰에서는 2004년 당시에는 공무원들도 선거 정치자금을 내는 것이 금지되지 않았었다고 주장했다. 하지만 오마이뉴스가 교과부에 확인한 바에 의하면 이는 사실과 다르다고 한다. 교과부 교원복무 담당자는 전화통화에서 교원의 정치자금 기부를 금지하는 규정은 1949년 제정된 이래 바뀐 적이 없었고 따라서 교사의 정치자금 기부가 합법이었던 적은 없었다고 밝혔다. 

top-1 passage with score 16.5452
빅히트 엔터테인먼트의 대표이자 프로듀서인 방시혁은 2010년 9월 2일, 힙합 그룹 방탄소년단의 새 멤버를 모집하는 전국 오디션을 개최한다고 밝혔다. 이 오디션은 포털사이트 다음과 함께 ‘힛잇(HIT IT)’이라는 타이틀로 진행 되었다. 이미 방탄소년단의 멤버로 발탁된 김남준에 대해 방시혁은 “언더 힙합신에서도 실력을 인정받은 고등학생 래퍼로 랩 메이킹에 탁월하며 프로 못지 않은 실력을 지니고 있다”고 소개했다.